In [0]:
#from google.colab import files
#files.upload()

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/My Drive/m3

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/m3


In [0]:
import numpy as np
import matplotlib.pyplot as plt
import literate
import numpy as np
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Lambda
from keras.models import load_model
import keras

syldict = literate.syldict("Syllable_dictionary.txt")
data, words = literate.read("shakespeare.txt", syldict)

Using TensorFlow backend.


In [0]:
def listwords_to_line(listwords):
    return " ".join(listwords)

def lines_to_sonnet(lines):
    return "\n".join(lines)

def listlist_to_sonnet(listlist):
    return lines_to_sonnet(list(map(listwords_to_line, listlist)))

def subthings(sonnet, n):
    a = []
    for i in np.arange(0,len(sonnet)-n):
        a.append(sonnet[i:i+n])
    return a

def sublabels(sonnet, n):
    a = []
    for i in np.arange(0,len(sonnet)-n):
        a.append(sonnet[i+n])
    return a

def flatten(a):
    return [item for sublist in a for item in sublist]

def split(word): 
    return [char for char in word]

def string_to_ints(s):
    return list(map(lambda x : char_to_int[x],split(s)))

def get_model(temp,n_lstm,X,y):
    model = Sequential()
    model.add(LSTM(n_lstm, input_shape=(X.shape[1], X.shape[2])))
    model.add(Lambda(lambda x: x / temp))
    model.add(Dense(vocab_size, activation='softmax'))
    return model

def train(model,n,use_adam,lr, X,y):
    #lr = 0.001
    adam = keras.optimizers.Adam(lr=lr, beta_1=0.9, beta_2=0.999, amsgrad=False)

    #lr =0.01
    sgd = keras.optimizers.SGD(lr=lr, momentum=0.0, nesterov=False)

    if(use_adam):
        opt = adam
    else:
        opt = sgd

    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    model.fit(X, y, epochs=n, verbose=2)
    return model

def gen(model, seed, n):
    k = len(seed)
    s = seed
    for i in range(n):
        s += predict_next(model,s[-k:])
    return s


In [0]:
def getXy(sonnets):
    listofsubsonnetlists = list(map(lambda x : subthings(x, s), sonnets))
    listoflabellists = list(map(lambda x : sublabels(x, s), sonnets))
    listofsubsonnets = flatten(listofsubsonnetlists)
    listoflabels = flatten(listoflabellists)
    intcharsubsonnets = list(map(string_to_ints, listofsubsonnets))
    intcharlabels = list(map(lambda x : char_to_int[x],listoflabels))
    X = intcharsubsonnets
    y = intcharlabels
    sequences = list(map(intstring_to_encoding,X))
    X = np.array(sequences)
    y = to_categorical(y, num_classes=vocab_size)
    return X,y

In [0]:

def intstring_to_encoding(su):
    return to_categorical(su, num_classes=vocab_size)

def string_to_encoding(s):
    return intstring_to_encoding(string_to_ints(s))

def predict_next(m,s):
    encoded = np.array([string_to_encoding(s)])
    yhats = m.predict(encoded, verbose=0)[0]
    return np.random.choice(chars,p=yhats)

In [0]:
def do(lr,n,use_adam,n_lstm,temp,name,X,y):
    model = get_model(temp,n_lstm, X,y)
    model = train(model,n,use_adam,lr,X,y)
    model.save(name)
    return model

def execp(p,X,y):
    return do(p[0],p[1],p[2],p[3],p[4],p[5],X,y)

In [0]:
def read(fn):
    f = open(fn,"r")
    contents = f.read()
    return contents

In [0]:
def gen(pf, seed, n):
    k = len(seed)
    s = seed
    for i in range(n):
        s += pf(s[-k:])
    return s

def is_prefix(p,s):
    return s.startswith(p)

def is_valid_so_far(word,words):
    for w in words:
        if is_prefix(word, w):
            return True
    return False

#seed ends in a space
def add_next_word(model, start, words, k, max_i):
    i=0
    while True:
        s=start
        w=""
        while True:
            i+=1
            if i > max_i:
                return "blah"
            p = predict_next(model,s[-k:])
            s += p
            if not is_valid_so_far(w,words):
                print(w)
                break
            if p == " " or p == "\n":
               return s
            w += p

def cleanup(s):
    s2 = ""
    for c in s:
        lc = c.lower()
        if lc in chars:
            s2 += lc   
    return s2

!pip install pronouncing
import pronouncing
def get_rhymes(word):
    return list(map(cleanup,pronouncing.rhymes(word)))

def syllables(word):
    count = 0
    vowels = 'aeiouy'
    word = word.lower()
    if word[0] in vowels:
        count +=1
    for index in range(1,len(word)):
        if word[index] in vowels and word[index-1] not in vowels:
            count +=1
    if word.endswith('e'):
        count -= 1
    if word.endswith('le'):
        count += 1
    if count == 0:
        count += 1
    return count

import nltk
nltk.download('cmudict')
from nltk.corpus import cmudict
d = cmudict.dict()

def num_syllables(word):
    try:
        return ([len(list(y for y in x if y[-1].isdigit())) for x in d[word.lower()]])[0]
    except KeyError:
        #if word not found in cmudict
        return syllables(word)

import random
def random_word_from(w,words):
    halfvalid = get_rhymes(w)
    valid = list(np.intersect1d(np.array(halfvalid,dtype="object"), np.array(list(words),dtype="object")))
    if len(valid)==0:
        if len(halfvalid)==0:
            return random.sample(words,1)[0]
        else:
            return random.sample(halfvalid,1)[0]
    return random.sample(valid,1)[0]

def add_next_rhyming_word(pf, start, k, words, max_i, rhyme, must_rhyme):
    i=0
    while True:
        s=start
        w=""
        while True:
            i+=1
            if i > max_i:
                w = random_word_from(rhyme, words)
                print(w)
                return "{}{}\n".format(start,w), w
            p = pf(s[-k:])#predict_next(model,s[-k:])
            s += p

            if p == " ":
                if not w in words:
                    break
                if must_rhyme:
                    break
                return s, w
            if p == "\n" and rhyme == "":
                if not w in words:
                    break
                return s, w
            if p == "\n":
                if not w in words:
                    break
                if w in get_rhymes(rhyme):
                    return s, w
                else:
                    break
            w += p
            if not is_valid_so_far(w,words):
                break

def sensegen(model, seed, nw, words, max_i):
    k=len(seed)
    s = seed
    for i in range(nw):
        s = add_next_word(model, s, words, k, max_i)
    return s

def rhymegen(pf, seed, lines, words, max_i, firstrhyme,first_line, numsylreq):
    none = ""
    endword1=firstrhyme
    endword2=""
    k=len(seed)
    s = seed
    line = first_line
    numsyl=0
    while True:
        if line >= lines:
            break
        rhyme = [none, none, endword2, endword2, none, none, endword2, endword2, none,none,endword2, endword2, none, endword1][line]
        if numsyl >= numsylreq-1:
            s,w = add_next_rhyming_word(pf, s, k, words, max_i, rhyme, True)
        else:
            s,w = add_next_rhyming_word(pf, s, k, words, max_i, rhyme, False)
        numsyl+=num_syllables(w)
        
        if s[-1] == "\n":
            print(line)
            line += 1
            endword2 = endword1
            endword1 = w
            numsyl=0
        
    return s

basicseed = "e\nshall i compare thee to a summers day\n"
def test_model(pf, words):
    normal = gen(pf, basicseed,1000)
    rhyme = rhymegen(pf, basicseed, 14, words, 1000,"day",1,10)
    print(normal)
    print(rhyme)

def pf(model):
    def pff(x):
        return predict_next(model,x)
    return pff

def prob(p):
    return random.uniform(0, 1) < p

def pf2(m1,m2,p):
    def pf2f(x): 
        if prob(p): 
            return predict_next(m1,x) 
        else:
            return predict_next(m2,x) 
    return pf2f

[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


In [0]:
sonnets = list(map(listlist_to_sonnet, data))
onestring="".join(sonnets)
chars=list(np.sort(np.unique(split(onestring))))
vocab_size = len(chars)
ints = range(len(chars))
char_to_int = dict(zip(chars, ints))
int_to_char = dict(zip(ints, chars))

s=40

coro = read("coro.txt")
coro2 = cleanup(coro)

XS,yS = getXy(sonnets)
XC,yC = getXy([coro2])

import re

wordsS = list(words)
wordsC = list(set(re.split(" |\n",coro2)))
wordsC.remove("")
wordsC.remove("e")
wordsC.remove("m")
wordsC.remove("n")
wordsC.remove("s")
wordsC.remove("b")
wordsSC = list(np.union1d(np.array(wordsS,dtype="object"),np.array(wordsC,dtype="object")))

In [0]:
norm = [0.001,50,True,100,1,"norm.h5"]
cor1 = [0.001,50,True,100,1,"cor1.h5"]
tune = [0.05,45,False,150,1,"tune.h5"]
tp25 = [0.001,50,True,100,0.25,"tp25.h5"]
tp75 = [0.001,50,True,100,0.75,"tp75.h5"]
t1p5 = [0.001,50,True,100,1.5,"t1p5.h5"]

norm_m = load_model("norm.h5")
#coro_m = load_model("cor1.h5")
#tp25_m = load_model("tp25.h5")
#tp75_m = load_model("tp75.h5")
#t1p5_m = load_model("t1p5.h5")
#tune_m = load_model("tune_m.h5")












Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




In [0]:
#norm_m = execp(norm,XS,yS)
coro_m = execp(cor1,XC,yC)
tp25_m = execp(tp25,XS,yS)
tp75_m = execp(tp75,XS,yS)
t1p5_m = execp(t1p5,XS,yS)
tune_m = execp(tune,XS,yS)

Epoch 1/50
 - 22s - loss: 2.7778 - acc: 0.1892
Epoch 2/50
 - 22s - loss: 2.3383 - acc: 0.3283
Epoch 3/50
 - 22s - loss: 2.1719 - acc: 0.3562
Epoch 4/50
 - 22s - loss: 2.0681 - acc: 0.3873
Epoch 5/50
 - 22s - loss: 1.9811 - acc: 0.4140
Epoch 6/50
 - 23s - loss: 1.9047 - acc: 0.4428
Epoch 7/50
 - 22s - loss: 1.8349 - acc: 0.4673
Epoch 8/50
 - 22s - loss: 1.7663 - acc: 0.4884
Epoch 9/50
 - 22s - loss: 1.7037 - acc: 0.5104
Epoch 10/50
 - 22s - loss: 1.6456 - acc: 0.5272
Epoch 11/50
 - 22s - loss: 1.5895 - acc: 0.5415
Epoch 12/50
 - 22s - loss: 1.5407 - acc: 0.5575
Epoch 13/50
 - 22s - loss: 1.4904 - acc: 0.5704
Epoch 14/50
 - 22s - loss: 1.4422 - acc: 0.5868
Epoch 15/50
 - 22s - loss: 1.3973 - acc: 0.5967
Epoch 16/50
 - 22s - loss: 1.3520 - acc: 0.6133
Epoch 17/50
 - 22s - loss: 1.3078 - acc: 0.6239
Epoch 18/50
 - 22s - loss: 1.2697 - acc: 0.6338
Epoch 19/50
 - 22s - loss: 1.2271 - acc: 0.6489
Epoch 20/50
 - 22s - loss: 1.1903 - acc: 0.6557
Epoch 21/50
 - 22s - loss: 1.1514 - acc: 0.6659
E

In [0]:
test_model(pf(norm_m), wordsS)

1
2
3
4
5
above
6
7
8
9
earth
10
11
12
13
e
shall i compare thee to a summers day
or vatulateress and from thee do i hold
that by youth thear his swarvang madest
when strite repexv
on both dost blooker his against the sight
for all that it in hold that bears shakerd
truth doth life is must motell in their doth
some shall'st fills ty less to knowted treasure
their out allowed oo praise in there time
to make a so to home what hoe to did still
and you grads beaters to he clulld but that tell
to more fronger's frest hand that you lack's old
why nor when it why boot the hoprs to wring
and gives thy sun so far let more orced
his strewex a somet then should more truth love to night
do remember to-s coptents amond a blon
now in you speak new feed thy sweet best
of trust the growne frave make tould like him ment
me in the could perectuince carring
shell bear wearetts of this rices the self
doth naked mine eye is love am your dead
the perfatequst natt repecrices summery
or chusts of you more pra

In [0]:
test_model(pf(coro_m), wordsC)

than
1
may
2
can
3
appearance
4
5
perseverance
6
form's
7
8
9
antivirus
10
high
11
garland
12
carland
13
e
shall i compare thee to a summers day
 repatec transmas incel menis coronavirus that betucated rna pridemiry animals actubiallye
to the aushing coronaviruses from a chimmone
alund a pronitis it ot -eprecciss of the coronavirus subficied cav regina
coronavirus
s kn trespriated to the surface protoby this virus caisivars bats of these animal cis-acting elements be ateh aldecad
the world respiratory hoth hulan to as be coronavirus  diver id a semse anchia cats be to the packeging from the nastare indy bevl-covar deveaton crown rearry
the gast cis-acting elements dine tial cause and bovididaly entymal and tracknow rastrondinting thenmmmicranily an  covorerienting the viral sadie and
a proptingent fromes arien fommstrompteminalitie otichulal bat reals which went human coronavirus causes a labge a mbcy repticatily ty aroun ueddalse tary
the other rna if it the bocat translated comoniris

In [0]:
test_model(pf(tp25_m), wordsS)

1
2
sought
3
4
5
attend
6
7
8
9
'tis
10
stage
11
12
13
e
shall i compare thee to a summers day
and healthful-sund i have love to how
thou mayst correwing wanting hours with show
and ten times had 'tan what in dost night
for that try another i whom slow outw
if thou shouldst be many hearth the expersed
to eyes sovere's mulat pleace the beauty still
but they sink the day truly out the kept
and may of you doth tell of mastallost
sporn distailent not excence wretched deark
my healt than their pripide is turns and change
still art my mind when picture so ron
thy ridety day tiken from whiche heit say
nifeing stoly self no untruddest
whell trow for that where although that prove compeate
but who hors thou hart me nor my cherals
and labe then be deaterness it groad
when i abuse the world's despective thee
ubustingitions tream hor prody
that you if how unlessed waste so of all
trink proinesth every where that love is
even there fors theref-can me hate eyes
can bearny me unfoot being wrinkless u

In [0]:
test_model(pf(tp75_m), wordsS)

1
nay
2
asked
3
4
5
making
6
yore
7
8
9
10
11
12
13
e
shall i compare thee to a summers day
givious prinl in jewel is fair grace
my love consumed in leave what canel show do plove
whilst i fave thou dusked child of these
and say mine own leanness with trust lines
and supting on thy breast doth leaves prove
as o'e words and to sparter madies of that straiswan
appatien rist toogmers tongrested
and many garmont that grace h's soppy fari
he to it my midds will pride shoulds spury
no nare ictes that is with those thought
if had fair abuses which have to be time
most durrer to mights of this behold away
sein it my best to his spire of wrong
but to make a tontless him thou minate
thy love that main's unthrift thy upprime
pownits theough thou thricls aftient new
und you is not live each that stars to lost
wilt three are d thie as all hesest love
a pentious though pen in infuncest of thine receaply
now not revel falther that sat mern's love
but thou be a vownion a tomb and
she bors of d arts my

In [0]:
test_model(pf(t1p5_m), wordsS)

1
2
guest
3
4
5
6
own
7
8
9
whereupon
10
sea
11
12
record
13
e
shall i compare thee to a summers day
upon them speed had in thy from mine eye bethe to thee my disdmadees are good
from might with in onumest puryom well
bean stesfeved then it seem doth goow kwefe
themselves this summers rose sum is not to be
roungle canganse that iver is pleasess sbent
soul and most made worthy worth o'eral endly
with wrille's but hath whose fauchered fadon
thy selves for me summers tomm please heod
at nex as book which threver i must tell
no glat seem from heay did and sipsuriest
for nave that for wouldst tober compound
a mad not which sold will so mide of my despite
the worst not apond that canning a pain
when yeters antime as my sad to lemses
one of that which love and provist the sweet self despise
and hate's reaur wrack dapking on thought
then world tingot to more thy sweet-self
and fray and streager bare this rue will more from find
be all meny gacks from love's vaines
o no longer no more in this m

In [0]:
test_model(pf(tune_m), wordsS)

1
gay
2
3
4
5
taught
6
7
8
9
ay
10
11
12
parts
13
e
shall i compare thee to a summers day
from for nawe rabe radest would than will spenth
orfumps in thine injenched undo in merous
byoy unceming oloned but wronk love astway
partesinj more thou beast of bllive apay
no morver as love all i might wis so
citnease flowed are hours yit hunghimn pride
whelh bainto heresionle gave die do conveing
rude in your acad ot love a frown'st
or inle will not lave home of you is my ride
swake rrom marge grown to true you gracty
distioved unwerimist fimmer that my friend
and for her mine own a'wious thruvelss cord
what doth mammed thoughtsway hath my more
thun iwpart we here touged wormowhidging breed
baly widiond hamoored beauty's might sers
but wis brass as art griphmle conding still of abe
as it you bothered emamed will state
to gany of rownob of me howersed
while i not sweet do i amight herwers
'tarks nother byes''st commaterbowide
o-liring the toryisgow putcoomed thine
than furterure with tan aclies

In [0]:
test_model(pf2(norm_m, coro_m,0.35), wordsSC)

1
2
3
4
5
gold
6
grace
7
8
9
species'
10
ministry
11
12
sore
13
e
shall i compare thee to a summers day
 omm bat autaly's dises that i sporsed diseases fron cate  lays
that from lifer  wn-etid lesh risgrins a bosease criamed
the syow riplit fypt aconater coomel
thich is mibcreastin of the fantle rambly cause husease clieks are genure the strain coround bile wna malsed
 nornipsiss beningged from the the ot a dumitiny the lucke pasion
int can bevely invons of subeation awo hume shrem con cluted hemats inelote
the muchinaly by a mort respirate
he is coumetye
comong blasen altong wo beugantls ousine nage that canter in offerd for cortseminaled
thy hearter to puth hogge mentuth namke repend
i sere convities of thy the larcr of coused in me
corong rosticilage  uscover  pass is domed easer
this in the serface for symed in cellicly
in of the coronavilusy hku-severaligangy
the colodavilus ypences
trongurlicions as who sice rich but healoned
the fears to trews us the swrugklupney
to new truts wi

In [0]:
"e" in wordsC

True

In [0]:
for c in "abcdefghijklmnopqrstuvwxyz":
  if c in wordsC:
    print(c)

a
b
m
n
s
